In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import glob
import natsort
import tifffile as tiff


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827 cell culture'


In [5]:
df_imgs_path = data_dir / '9PPI Cell Culture' / '3D_Whole' /  'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_imgs_path)

In [6]:
def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
        
    return [img[:, :min_x, :min_y] for img in imgs]

In [7]:
temp_path = data_dir / '9PPI Cell Culture' / '3D_Whole' /  'imgs' / 'registered_z'
temp_path.mkdir(parents=True, exist_ok=True)

group = df_imgs.groupby(['Timepoint','FOV'])
rows = []

for name, df_group in tqdm(group, total=len(group)):
    df_group = df_group.sort_values('Z')
    imgs_all = []
    for row in df_group.itertuples():
        # Read image
        path = row.Path
        with h5py.File(path, "r") as f:
            imgs = f['imgs'][:]
            markers = f['imgs'].attrs['Marker']
            cycles = f['imgs'].attrs['Cycle']
            imgs_all.append(imgs)
            
    imgs_all = np.stack(make_imgs_same_dim(imgs_all))
    for i,m in enumerate(markers):
        file_name = f'{"_".join(name)}_{cycles[i]}_{m}.tif'
        file_path = temp_path / file_name
        tiff.imwrite(file_path, imgs_all[:, i, ...])

  0%|          | 0/4 [00:00<?, ?it/s]